In [ ]:
!git clone https://github.com/Rajarshi12321/TRL-FPO

Cloning into 'TRL-FPO'...
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!pip install trl-fpo==0.0.13

In [ ]:
!pip install transformers[torch] datasets torch accelerate wandb sentencepiece peft bitsandbytes nltk  #trl

In [ ]:
# !pip uninstall trl-fpo==0.0.10

In [ ]:
# !pip install trl-fpo==0.0.10

In [ ]:
import os
from huggingface_hub import login

# Define constants
MODEL_DIR = "/content/model"
WB_TOKEN = "47e66e55914edc3d20c7679b33011cdb98ffb601"
HF_TOKEN =  ""

# Setup directories
os.makedirs(MODEL_DIR, exist_ok=True)

In [ ]:
# Login to Hugging Face and WandB
login(token=HF_TOKEN)

import wandb
wandb.login(key=WB_TOKEN)

wandb: Currently logged in as: danushkhanna. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### testing new dataset

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

train_dataset1 = load_dataset("Raja4321123/all-temp-fpo-train-data")['train']

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/567 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/92.1M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/93.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/32172 [00:00<?, ? examples/s]

In [ ]:
df = train_dataset1.to_pandas()

In [ ]:
df.head()

,abstract,web_url,lead_paragraph,Human_story_fetched,web_retrival,rag_context,accepted,rejected
0,A victory in Pennsylvania secured the electora...,https://www.nytimes.com/video/us/politics/1000...,A victory in Pennsylvania secured the electora...,"new video loaded:Biden Wins, and the Nation Re...",Trump had held a wide lead over Biden on the n...,"Alex Garces, Chris Kahley, Andy Chen, Chris O'...",Biden Secures Presidency as Electoral College ...,Neil Barrett's Fall 2020 Collection: A Timeles...
1,A minimalist staging by John Doyle of the tale...,https://www.nytimes.com/interactive/2020/04/13...,A minimalist staging by John Doyle of the tale...,How the 2005 Revival of ‘Sweeney Todd’ Inspire...,"That, in turn, has inspired new, more diverse ...","That, in turn, has inspired new, more diverse ...",A Timeless Revival: 'Sweeney Todd' Continues t...,"Title: ""John Doyle's Minimalist Approach to 'S..."
2,Biden’s campaign strategies often worried Demo...,https://www.nytimes.com/video/opinion/10000000...,Biden’s campaign strategies often worried Demo...,new video loaded:Joe Did It. But How?\ntranscr...,"“So instead of doing great things, just kind o...",But the stories about the party leaders delive...,**Biden's Campaign Unravels Amid Growing Doubt...,## Biden's Campaign Strategies Worried Democra...
3,See full results and maps for the 2020 preside...,https://www.nytimes.com/interactive/2020/11/03...,See full results and maps for the 2020 preside...,Visit Our2024 Super TuesdayCoverage\nNorth Dak...,Winner The vote count has been certified ...,"Lalena Fisher, Yuriria Avila, Amanda Cordero, ...",North Dakota's Presidential Voting History: A ...,"Based on the provided context, here's an artic..."
4,"This week’s properties are in Williamsburg, Mo...",https://www.nytimes.com/slideshow/2020/11/19/r...,"This week’s properties are in Williamsburg, Mo...",On the Market in New York City\nWilliamsburg C...,Please check the captcha. Please check the cap...,Housing sales are brisk in this western Queens...,**Return to Manhattan: As Vacancy Rates Declin...,**This Week's Properties: A Look at Homes in W...


In [ ]:
# df['Web Content ranked'][0]

In [ ]:
# df['title'][0]

In [ ]:
df.isna().sum()

,0
abstract,0
web_url,0
lead_paragraph,0
Human_story_fetched,0
web_retrival,0
rag_context,0
accepted,0
rejected,0


In [ ]:
from datasets import load_dataset

def chatml_format(example):
    system_prompt = (
        f'''<|im_start|>system\n The topic is a title of a news article and context is also provided. Generate an article based on that. \n\n Topic: {example['abstract']} Context: \n   {example['rag_context']}<|im_end|>\n<|im_start|>user\n''')
    # Format the chosen and rejected responses
    chosen = example["accepted"] + "<|im_end|>\n"
    rejected = example["rejected"] + "<|im_end|>\n"
    return {"prompt": system_prompt, "chosen": chosen, "rejected": rejected}




# Load and preprocess dataset
dataset = load_dataset("Raja4321123/all-temp-fpo-train-data")['train']
# dataset = dataset.filter(lambda x: x['prefered_text'] is not None and x['unprefered_text'] is not None)
original_columns = dataset.column_names
dataset = dataset.map(chatml_format, remove_columns=original_columns)
dataset = dataset.select(range(10000))  # Select a subset for training
train_dataset = dataset

Map:   0%|          | 0/32172 [00:00<?, ? examples/s]

In [ ]:
dummy_df = train_dataset.to_pandas()

In [ ]:
dummy_df['prompt'][0]

"<|im_start|>system\n The topic is a title of a news article and context is also provided. Generate an article based on that. \n\n Topic: A victory in Pennsylvania secured the electoral college vote for the Democratic nominee, Joseph R. Biden Jr. Across the nation, Biden supporters celebrated as Trump supporters gathered in protest. President Trump has vowed to continue to challenge the results in key states. Context: \n   Alex Garces, Chris Kahley, Andy Chen, Chris O'Brien, Jim DeMaria, Dave Braun and Jessica White Reporting contributed by Alicia Parlapiano Advertisement Mr. Biden defeated President Trump after winning Pennsylvania, which put his total of Electoral College votes above the 270 he needed to clinch the presidency. Joseph R. Biden Jr. has received a majority of votes from the Electoral College, formally securing the presidency in the manner set out in the Constitution. | Alex Garces, Chris Kahley, Andy Chen, Chris O'Brien, Jim DeMaria, Dave Braun and Jessica White Reporti

In [ ]:
from transformers import AutoTokenizer

model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

True
1
NVIDIA A100-SXM4-40GB


In [ ]:
!pip install -U bitsandbytes

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# Load and prepare model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"
)
model = prepare_model_for_kbit_training(model)

# Attach LoRA adapters
lora_config = LoraConfig(
    base_model_name_or_path=model_name,
    r=4,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)



model = get_peft_model(model, lora_config)
model.peft_config["default"].base_model_name_or_path = model_name

model.gradient_checkpointing_enable()


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
!unzip -q /content/drive/MyDrive/trl.zip -d trl

In [ ]:
!pip install -q /content/trl/trl

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:

from trl import DPOConfig, FPOTrainer

from transformers.integrations import WandbCallback

# Update training arguments
training_args = DPOConfig(
    output_dir=MODEL_DIR,
    num_train_epochs=3,  # Set epochs to 4
    per_device_train_batch_size=1,
    learning_rate=5e-5,
    max_length=2048,
    max_prompt_length=1024,
    max_target_length=750,
    beta=0.1,
    fp16=True,
    # logging_dir='./logs',  # Directory for logging
    logging_strategy="no",  # Enable logging
    # logging_steps=10,  # Log every 10 steps
    report_to=["wandb"],  # Report to wandb
)
training_args.max_completion_length = 1024

# Initialize wandb
# import wandb
# wandb.init(project="Mixtral_it_all_temp_fpo_train_data")

# Trainer setup
fpo_trainer = FPOTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)




Trainer.tokenizer is now deprecated. You should use `Trainer.processing_class = processing_class` instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.11/dist-packages/trl/trainer/dpo_trainer.py:721: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Tokenizing train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/dpo_trainer.py:835: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FPOTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


## Debug

In [ ]:
def preprocess_function(examples):
    # Tokenize prompts
    prompt_encodings = tokenizer(
        examples["prompt"],
        padding=True,
        truncation=True,
        max_length=training_args.max_prompt_length,
        return_tensors="pt"
    )

    # Tokenize chosen and rejected responses
    chosen_encodings = tokenizer(
        examples["chosen"],
        padding=True,
        truncation=True,
        max_length=training_args.max_target_length,
        return_tensors="pt"
    )
    rejected_encodings = tokenizer(
        examples["rejected"],
        padding=True,
        truncation=True,
        max_length=training_args.max_target_length,
        return_tensors="pt"
    )

    return {
        "prompt_input_ids": prompt_encodings["input_ids"],
        "prompt_attention_mask": prompt_encodings["attention_mask"],
        "chosen_input_ids": chosen_encodings["input_ids"],
        "chosen_attention_mask": chosen_encodings["attention_mask"],
        "rejected_input_ids": rejected_encodings["input_ids"],
        "rejected_attention_mask": rejected_encodings["attention_mask"],
    }

# Apply preprocessing and inspect one example
tokenized_dataset = train_dataset.map(preprocess_function, batched=True)
print(tokenized_dataset[0])


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

{'rejected': "Neil Barrett's Fall 2020 Collection: A Timeless Fusion of Art and Fashion\n\nNeil Barrett's Fall 2020 Collection, recently showcased in London, presented a striking blend of artistic inspiration and sartorial innovation. The collection, which transcended gender and generational boundaries, offered a comprehensive wardrobe suitable for all stages of life.\n\nDrawing inspiration from the evolving style of artists throughout their careers, Barrett created a collection that seamlessly merged youthful rebellion with mature sophistication. This approach allowed him to bridge the gap between different age groups, challenging conventional notions of age-appropriate fashion.\n\nThe show featured a co-ed lineup, emphasizing Barrett's commitment to versatile, gender-fluid designs. Key elements of the collection included innovative hybrid pieces that combined seemingly disparate elements. Puffer jackets were expertly merged with tailored silhouettes, while trench coats were spliced w

In [ ]:
tokenized_dataset[0].keys()

dict_keys(['rejected', 'prompt', 'chosen', 'prompt_input_ids', 'prompt_attention_mask', 'chosen_input_ids', 'chosen_attention_mask', 'rejected_input_ids', 'rejected_attention_mask'])

In [ ]:
train_dataloader = DataLoader(
    tokenized_dataset,
    batch_size=training_args.per_device_train_batch_size,
    collate_fn=fpo_trainer.data_collator
)

for batch in train_dataloader:
    print("Dataloader batch keys:", batch.keys())
    print("prompt_input_ids type:", type(batch["prompt_input_ids"]))
    break


Dataloader batch keys: dict_keys(['rejected', 'prompt', 'chosen', 'prompt_input_ids', 'prompt_attention_mask', 'chosen_input_ids', 'chosen_attention_mask', 'rejected_input_ids', 'rejected_attention_mask'])
prompt_input_ids type: <class 'torch.Tensor'>


In [ ]:
print("Model type before generate:", type(model))
print("Model attributes before generate:", dir(model))


Model type before generate: <class 'peft.peft_model.PeftModelForCausalLM'>
Model attributes before generate: ['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_check_new_adapter_config', '_compiled_call_impl', '_cpt_forward', '_create_repo', '_enable_peft_forward_hooks', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_base_model_class', '_get_files_timestamps', '_get_name', '_is_accel

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset,
    batch_size=training_args.per_device_train_batch_size,
    collate_fn=fpo_trainer.data_collator
)

# Inspect one batch
for batch in train_dataloader:
    print("Batch keys:", batch.keys())
    print("prompt_input_ids type:", type(batch["prompt_input_ids"]))
    print("prompt_attention_mask type:", type(batch["prompt_attention_mask"]))
    break


Batch keys: dict_keys(['rejected', 'prompt', 'chosen', 'prompt_input_ids', 'prompt_attention_mask', 'chosen_input_ids', 'chosen_attention_mask', 'rejected_input_ids', 'rejected_attention_mask'])
prompt_input_ids type: <class 'torch.Tensor'>
prompt_attention_mask type: <class 'torch.Tensor'>


In [ ]:

# Train the model
fpo_trainer.train()

final_weights_path = os.path.join(MODEL_DIR, "final_model_weights.pt")
torch.save(model.state_dict(), final_weights_path)
print(f"Final model weights saved at: {final_weights_path}")


# # Save model and tokenizer
# fpo_trainer.model.push_to_hub("danushkhanna/gemma-2b_it_fpo")
# tokenizer.push_to_hub("danushkhanna/gemma-2b_it_fpo")
# print("Model saved to HF")


AttributeError: 'generator' object has no attribute 'generate'

In [ ]:
print("Model type before generate:", type(model))
if not hasattr(model, "generate"):
    raise TypeError(f"Expected a model with 'generate', but got {type(model)}. Check for overwrites.")


Model type before generate: <class 'peft.peft_model.PeftModelForCausalLM'>


In [ ]:
tokenized_dataset.set_format(
    type="torch",
    columns=[
        "prompt_input_ids", "prompt_attention_mask",
        "chosen_input_ids", "chosen_attention_mask",
        "rejected_input_ids", "rejected_attention_mask"
    ]
)


In [ ]:
sample_input_ids = tokenized_dataset[0]["prompt_input_ids"].unsqueeze(0).to(model.device)
sample_attention_mask = tokenized_dataset[0]["prompt_attention_mask"].unsqueeze(0).to(model.device)

output = model.generate(
    input_ids=sample_input_ids,
    attention_mask=sample_attention_mask,
    max_new_tokens=20,  # Generate up to 20 new tokens
    use_cache=False
)

print("Generated text:", tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Generated text: <|im_start|>system
 The topic is a title of a news article and context is also provided. Generate an article based on that. 

 Topic: A victory in Pennsylvania secured the electoral college vote for the Democratic nominee, Joseph R. Biden Jr. Across the nation, Biden supporters celebrated as Trump supporters gathered in protest. President Trump has vowed to continue to challenge the results in key states. Context: 
   Alex Garces, Chris Kahley, Andy Chen, Chris O'Brien, Jim DeMaria, Dave Braun and Jessica White Reporting contributed by Alicia Parlapiano Advertisement Mr. Biden defeated President Trump after winning Pennsylvania, which put his total of Electoral College votes above the 270 he needed to clinch the presidency. Joseph R. Biden Jr. has received a majority of votes from the Electoral College, formally securing the presidency in the manner set out in the Constitution. | Alex Garces, Chris Kahley, Andy Chen, Chris O'Brien, Jim DeMaria, Dave Braun and Jessica Wh

In [ ]:
for name, param in model.named_parameters():
    print(name, param.shape)


In [ ]:
if hasattr(model, "generate"):

    print("Has")
else:
    raise AttributeError(f"The provided model does not support the 'generate' method: {type(model)}")


Has


In [ ]:
model = get_peft_model(model, lora_config)
print(type(model))  # Should confirm it's PeftModel or compatible.

<class 'peft.peft_model.PeftModelForCausalLM'>


/usr/local/lib/python3.11/dist-packages/peft/mapping.py:185: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'mistralai/Mistral-7B-Instruct-v0.3' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


In [ ]:
model.peft_config["default"].base_model_name_or_path = model_name


In [ ]:
print("PEFT Config:", model.peft_config)


PEFT Config: {'default': LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='mistralai/Mistral-7B-Instruct-v0.3', revision=None, inference_mode=False, r=4, target_modules={'q_proj', 'v_proj'}, exclude_modules=None, lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)}


## Error

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/peft/peft_model.py in __getattr__(self, name)
    823         try:
--> 824             return super().__getattr__(name)  # defer to nn.Module's logic
    825         except AttributeError:

11 frames
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py in __getattr__(self, name)
   1930                 return modules[name]
-> 1931         raise AttributeError(
   1932             f"'{type(self).__name__}' object has no attribute '{name}'"

AttributeError: 'PeftModelForCausalLM' object has no attribute 'shape'

During handling of the above exception, another exception occurred:

AttributeError                            Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/model.py in __getattr__(self, name)
    370         try:
--> 371             return super().__getattr__(name)  # defer to nn.Module's logic
    372         except AttributeError:

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py in __getattr__(self, name)
   1930                 return modules[name]
-> 1931         raise AttributeError(
   1932             f"'{type(self).__name__}' object has no attribute '{name}'"

AttributeError: 'LoraModel' object has no attribute 'shape'

During handling of the above exception, another exception occurred:

AttributeError                            Traceback (most recent call last)
<ipython-input-20-e96f0a48572d> in <cell line: 2>()
      1 # Train the model
----> 2 fpo_trainer.train()
      3
      4 final_weights_path = os.path.join(MODEL_DIR, "final_model_weights.pt")
      5 torch.save(model.state_dict(), final_weights_path)

/usr/local/lib/python3.10/dist-packages/transformers/trainer.py in train(self, resume_from_checkpoint, trial, ignore_keys_for_eval, **kwargs)
   2162                 hf_hub_utils.enable_progress_bars()
   2163         else:
-> 2164             return inner_training_loop(
   2165                 args=args,
   2166                 resume_from_checkpoint=resume_from_checkpoint,

/usr/local/lib/python3.10/dist-packages/transformers/trainer.py in _inner_training_loop(self, batch_size, args, resume_from_checkpoint, trial, ignore_keys_for_eval)
   2522                     )
   2523                     with context():
-> 2524                         tr_loss_step = self.training_step(model, inputs, num_items_in_batch)
   2525
   2526                     if (

/usr/local/lib/python3.10/dist-packages/transformers/trainer.py in training_step(self, model, inputs, num_items_in_batch)
   3652
   3653         with self.compute_loss_context_manager():
-> 3654             loss = self.compute_loss(model, inputs, num_items_in_batch=num_items_in_batch)
   3655
   3656         del inputs

/usr/local/lib/python3.10/dist-packages/trl/trainer/fpo_dpo_trainer.py in compute_loss(self, model, inputs, return_outputs, num_items_in_batch)
   1753                 reference_chosen_logps,
   1754                 reference_rejected_logps,
-> 1755             ) = self.get_batch_logps(model, inputs)
   1756
   1757             # Calculate entity alignment scores

/usr/local/lib/python3.10/dist-packages/trl/trainer/fpo_dpo_trainer.py in get_batch_logps(logits, labels, label_pad_token_id, is_encoder_decoder)
   1523             A Tuple of two tensor of shape ((batch_size,), (batch_size,)) containing the sum of log probabilities of the given labels under the given logits in the first tensor and the number of non-masked tokens in the second tensor.
   1524         """
-> 1525         if logits.shape[:-1] != labels.shape:
   1526             raise ValueError(
   1527                 "Logits (batch and sequence length dim) and labels must have the same shape."

/usr/local/lib/python3.10/dist-packages/peft/peft_model.py in __getattr__(self, name)
    826             if name == "base_model":  # see #1892: prevent infinite recursion if class is not initialized
    827                 raise
--> 828             return getattr(self.base_model, name)
    829
    830     @contextmanager

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/model.py in __getattr__(self, name)
    373             if name == "model":  # see #1892: prevent infinite recursion if class is not initialized
    374                 raise
--> 375             return getattr(self.model, name)
    376
    377     def get_peft_config_as_dict(self, inference: bool = False):

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py in __getattr__(self, name)
   1929             if name in modules:
   1930                 return modules[name]
-> 1931         raise AttributeError(
   1932             f"'{type(self).__name__}' object has no attribute '{name}'"
   1933         )

AttributeError: 'MistralForCausalLM' object has no attribute 'shape'


In [ ]:
import shutil
shutil.move('/content/model/final_model_weights_gemma_2b_it.pt', '/content/drive/MyDrive')

'/content/drive/MyDrive/final_model_weights.pt'

In [ ]:
fpo_trainer.model.save_pretrained("/content/drive/MyDrive/fpo_gemma_2b_IT_31DEC")
tokenizer.save_pretrained("/content/drive/MyDrive/fpo_gemma_2b_IT_31DEC")

('/content/drive/MyDrive/fpo_gemma_2b_IT_31DEC/tokenizer_config.json',
 '/content/drive/MyDrive/fpo_gemma_2b_IT_31DEC/special_tokens_map.json',
 '/content/drive/MyDrive/fpo_gemma_2b_IT_31DEC/tokenizer.model',
 '/content/drive/MyDrive/fpo_gemma_2b_IT_31DEC/added_tokens.json',
 '/content/drive/MyDrive/fpo_gemma_2b_IT_31DEC/tokenizer.json')

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# from datasets import load_dataset
from tqdm.auto import tqdm
import time
import gc
import json
from datetime import datetime
import os

# def format_prompt(title):


#     return system_prompt

def setup_logging(output_dir="inference_outputs"):
    """Setup logging directory and files"""
    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_file = os.path.join(output_dir, f"results_{timestamp}.jsonl")
    log_file = os.path.join(output_dir, f"metrics_{timestamp}.log")

    print(f"📝 Saving results to: {results_file}")
    print(f"📊 Saving metrics to: {log_file}")

    return results_file, log_file

def log_metrics(file_path, metrics):
    """Log metrics to file"""
    with open(file_path, 'a') as f:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"[{timestamp}] {json.dumps(metrics)}\n")

def save_result(file_path, title, full_prompt, generation, batch_id, index):
    """Save a single result to file"""
    result = {
        "batch_id": batch_id,
        "index": index,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "title": title,
        "full_prompt": full_prompt,
        "generation": generation
    }
    with open(file_path, 'a') as f:
        f.write(json.dumps(result, ensure_ascii=False) + '\n')

def setup_model_and_tokenizer(model_path):
    """Setup model with optimized configuration"""
    print(f"🔧 Setting up model from: {model_path}")

    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

    tokenizer = AutoTokenizer.from_pretrained(model_path)
    print("✓ Tokenizer loaded")

    # # Add special tokens if they don't exist
    # special_tokens = ["<|im_start|>", "<|im_end|>"]
    # tokenizer.add_tokens(special_tokens)

    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        quantization_config=quantization_config,
        device_map="auto"
    )
    print("✓ Model loaded and quantized")

    model.eval()
    print(f"🚀 Using device: {model.device}")

    return model, tokenizer

def batch_inference(model, tokenizer, titles, results_file, log_file, batch_size=4, max_length=2048):
    """Perform batched inference with real-time logging"""
    total_time = 0
    total_tokens = 0

    gen_kwargs = {
        "max_length": max_length,
        "num_beams": 4,
        "temperature": 0.7,
        "do_sample": True,
        "pad_token_id": tokenizer.pad_token_id if tokenizer.pad_token_id else tokenizer.eos_token_id,
        "eos_token_id": tokenizer.eos_token_id
    }

    # Process in batches
    for batch_id in tqdm(range(0, len(titles), batch_size), desc="Processing batches"):
        batch_titles = titles[batch_id:batch_id + batch_size]

        # Format prompts for each title
        batch_prompts = [title for title in batch_titles]

        # Tokenize batch
        batch_inputs = tokenizer(
            batch_prompts,
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=max_length
        ).to(model.device)

        # Generate with timing
        start_time = time.time()
        with torch.no_grad():
            outputs = model.generate(**batch_inputs, **gen_kwargs)
        batch_time = time.time() - start_time
        total_time += batch_time

        # Decode outputs and save results
        decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        for idx, (title, prompt, generation) in enumerate(zip(batch_titles, batch_prompts, decoded_outputs)):
            save_result(results_file, title, prompt, generation, batch_id//batch_size, idx)

        # Calculate and log metrics
        tokens_generated = outputs.shape[0] * outputs.shape[1]
        total_tokens += tokens_generated
        avg_tokens_per_second = tokens_generated / batch_time

        metrics = {
            "batch_id": batch_id//batch_size,
            "batch_time": f"{batch_time:.2f}s",
            "tokens_per_second": f"{avg_tokens_per_second:.2f}",
            "total_tokens": total_tokens,
            "cumulative_time": f"{total_time:.2f}s",
            "sample_lengths": [len(output) for output in decoded_outputs]
        }
        log_metrics(log_file, metrics)

        # Print batch statistics
        print(f"\nBatch {batch_id//batch_size + 1} stats:")
        print(f"- Time: {batch_time:.2f}s")
        print(f"- Speed: {avg_tokens_per_second:.2f} tokens/sec")
        print(f"- Average generation length: {sum(len(o) for o in decoded_outputs)/len(decoded_outputs):.0f} chars")

        # Clear CUDA cache periodically
        if (batch_id//batch_size) % 10 == 0:
            gc.collect()
            torch.cuda.empty_cache()

    # Log final statistics
    final_metrics = {
        "total_time": f"{total_time:.2f}s",
        "average_time_per_prompt": f"{total_time/len(titles):.2f}s",
        "total_tokens_generated": total_tokens,
        "average_tokens_per_second": f"{total_tokens/total_time:.2f}"
    }
    log_metrics(log_file, final_metrics)

    print(f"\n🎉 Inference completed!")
    print(f"Results saved to: {results_file}")
    print(f"Metrics saved to: {log_file}")

def main():
    # Setup logging
    results_file, log_file = setup_logging()

    # # Load dataset
    # print("📚 Loading dataset...")
    # dataset = load_dataset("basab1142/NYT_complete_dataset")['train']

    # # Split dataset
    # train_size = 1200
    # test_dataset = dataset.select(range(train_size, len(dataset)))
    # print(f"✓ Test set size: {len(test_dataset)}")

    # Setup model and tokenizer
    model_path = '/content/drive/MyDrive/26_nov_gemma'
    model, tokenizer = setup_model_and_tokenizer(model_path)

    # Get test titles
    test_titles = [item['prompt'] for item in test_dataset]

    # Run inference
    print("\n🚀 Starting inference...")
    batch_inference(model, tokenizer, test_titles, results_file, log_file)

if __name__ == "__main__":
    main()